# __This notebook is used to create a Planet subscription for a specific geometry and download planetary variables data (Biomass and Soil Moisture).__
_____________________________________________________________
### A Python toolbox developed based on preliminary scripts supplied by Planet Inc. Credit is given to Planet Inc. for the initial code framework.
#### Developed by Dr. Amine Merzouki (LLD-STB, AAFC) - July 2025.
_____________________________________________________________

## 1. Import utility functions and librairies 

In [1]:
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import datetime
from datetime import datetime, timedelta
import numpy as np
from shapely.geometry import shape
import leafmap.foliumap as leafmap
from IPython.display import display
import geopandas as gpd
from planet import Planet, Session
from planet.clients import FeaturesClient
from planet.auth import APIKeyAuth
import planet.subscription_request
from sentinelhub import (
    SHConfig, SentinelHubRequest, CRS, MimeType, BBox, SentinelHubDownloadClient,
    SentinelHubRequest, DataCollection, MimeType, Geometry, CRS
)
from sentinelhub_downloader import SentinelHubAPI, Config


## 2. User Inputs 

In [2]:
# Collection ID of the Feature Collection with a single AOI, with quota reserved for the PV you're generating. 
PLANET_FEATURE_COLLECTION_ID = "ll_ns8_biomass-YqlpW4q"

# Your Planet API Key. You can get this from https://www.planet.com/account/#/user-settings 
# or if you have done `pip install planet` then you can type `planet auth value` and copy the results.
PLANET_API_KEY = "PLAK019816b0a5934143a123455d2fc4" #"<insert your key here>"

# Sentinel Hub Client ID and Secret. Go to https://apps.sentinel-hub.com/dashboard/#/account/settings
# and hit '+ Create' under OAuth Clients. Be sure to store these, you won't see 'secret' again.
SENTINEL_HUB_CLIENT_ID = "3em0d6125-1f57-4d4d-9a1b-12354c9b1297e" #"<insert your client id here>"
SENTINEL_HUB_CLIENT_SECRET = "6X9rS4RpnQWUXCEg1235487JCaKb5k" #"<insert your client secret here>"


# This name is used to identify the subscription, customize it to be a meaningful name
# so you can easily identify it later.
SUBSCRIPTION_BASE_NAME = "LL_NS8_Biomass"

# Customize these values for the PV you want to download.
#PV_VAR_TYPE="soil_water_content"  # PV type for Soil Water Content
#PV_VAR_ID="SWC-SMAP-L_V2.0_100" # SWC L band 100 meter 
# Other common options for SWC are: SWC-AMSR2-C_V2.0_100, SWC-AMSR2-X_V2.0_100, SWC-SMAP-L_V5.0_1000
# Full list of options: https://developers.planet.com/docs/planetary-variables/soil-water-content-technical-specification/

# These are the biomass proxy options.
PV_VAR_TYPE="biomass_proxy"  # PV type for Biomass Proxy
PV_VAR_ID="BIOMASS-PROXY_V4.0_10"

# Bands
#BANDS="SWC,SWC_MaskedPixels,QF"

# Start date for the data you want to download.
SUB_START_TIME = "2019-02-20"

# Comment out either an end date or NO_END_TIME to make an ongoing subscription
SUB_END_TIME = "2025-05-10"
#SUB_END_TIME = "NO_END_TIME"

# Define output folder for TIFF files
# Change this to the path of the folder you want to save the TIFF files to
OUTPUT_FOLDER = "./downloads/LL_NS_Biomass"

# Output template for the tiff files that result. The template supports the following variables
# {byoc_id} - The BYOC collection ID
# {date} - The date of the image
# {id} - A unique identifier for the image
OUTPUT_TEMPLATE = "LL_NS8_Biomass_{date}.tiff" 


## 3. Check to make sure user API key is set up and working

##### Should make a green checkmark after this cell if it set right

In [3]:
##defining base url
subscriptions_url = "https://api.planet.com/subscriptions/v1"

##authentication
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(subscriptions_url, auth=auth)
if response.status_code == 200:
    print("✅ Success - API Key is valid")
else:
    print(f"❌ Error: {response.status_code}, {response.text}")
    response.raise_for_status()

# Initialize Planet client
pl = Planet(session=Session(auth=APIKeyAuth(key=PLANET_API_KEY)))

✅ Success - API Key is valid


##### Get the geometry of your AOI from the features manager. Be sure you only have one AOI in the collection, if you don't then this will just select the first one.

In [4]:
features = list(pl.features.list_items(PLANET_FEATURE_COLLECTION_ID))# print number of items
print(f"Number of items: {len(features)}")

if len(features) > 1:
  print("❌ More than one item found, please check the collection and make sure it only contains a single AOI")
else:
  AOI_feature = features[0]
  print(f"✅ Using AOI from Features Manager: {AOI_feature['id']}: {AOI_feature['properties']}")

Number of items: 1
✅ Using AOI from Features Manager: NS-8: {'AOI': 'NS-8', 'pl:ref': 'pl:features/my/ll_ns8_biomass-YqlpW4q/ns-8-gjYnYYo', 'pl:area': 736560.0176823642}


### 4. Visualize the area uploaded to Planet on a map (Optional)
##### Give visual confirmation that your AOI is correct.

In [5]:
# Get centroid of first feature for map center
AOI_json_geometry = AOI_feature['geometry']
geom = shape(AOI_json_geometry)
centroid = geom.centroid

# Create the map
m = leafmap.Map(center=[centroid.y, centroid.x], zoom=8)

# Create a GeoJSON feature collection with the first X features
feature_collection = {
    "type": "FeatureCollection",
    "features": features
}

# Add the features to the map
m.add_geojson(
    feature_collection,
    layer_name="Features",
    style={
        "color": "blue",
        "fillColor": "#3366cc",
        "weight": 1,
        "fillOpacity": 0.3
    },
    hover_style={
        "fillOpacity": 0.7
    },
    info_mode='on_hover'
)

# Add basemap
m.add_basemap("OpenStreetMap")

print(f"Displaying AOI from Features Manager")
display(m)

Displaying AOI from Features Manager


### 5. Create a subscription for the AOI, and stores the subscription id and the data collection id. 
##### The data collection is the online collection of data that can be accessed through the Sentinel Hub API's. 

In [6]:
def create_planet_subscription(item):
    start_time = datetime.strptime(SUB_START_TIME, "%Y-%m-%d")
    
    subscription_name = f"{SUBSCRIPTION_BASE_NAME} - {item['id']}"
    
    source_params = {
        'start_time': start_time,
        'var_type': PV_VAR_TYPE,
        'var_id': PV_VAR_ID,
        'geometry': item['properties']['pl:ref']
    }
    
    if SUB_END_TIME != "NO_END_TIME":
        end_time = datetime.strptime(SUB_END_TIME, "%Y-%m-%d")
        source_params['end_time'] = end_time
    
    payload = planet.subscription_request.build_request(
        name=subscription_name,
        source=planet.subscription_request.planetary_variable_source(
            **source_params
        ),
        hosting=planet.subscription_request.sentinel_hub(
             collection_id="",
             create_configuration=True
        )
    )
    
    try:
        result = pl.subscriptions.create_subscription(payload)
        subscription_id = result["id"]
        data_collection_id = result["hosting"]["parameters"]["collection_id"]
        print(f"✅ Created subscription named '{subscription_name}'")
        print(f"   Subscription ID: {subscription_id}")
        print(f"   Data Collection ID: {data_collection_id}")
        return subscription_id, data_collection_id
    except Exception as e:
        print(f"❌ Error creating subscription '{subscription_name}': {str(e)}")
        raise

# Create subscriptions for each item
sub_id, coll_id = create_planet_subscription(AOI_feature)
data_collection_id = coll_id


✅ Created subscription named 'LL_NS8_Biomass - NS-8'
   Subscription ID: 599624a6-721a-49cf-86f7-373e72e7b26c
   Data Collection ID: bfdde5ec-f114-475c-9448-c5c580683e00


### 6. Wait for the subscription to complete

Now it's time to pause in this workflow and wait for all the data for the subscription to be generated. This usually takes a few hours, and if it's a big subscription it could take longer. You can run the following cell to check the status of the subscription and if you're getting results. If the status is 'pending' then it's not yet started, and is waiting for compute resources. If it says 'running' then data is being generated. You may be able to download some results during this time. If it says 'completed' then the data is definitely ready to download. Ongoing subscriptions will not say 'completed', but will still say running. So if the number of results is consistent then it's probably good to download. 


In [13]:

# print subscription status
subscription_response = pl.subscriptions.get_subscription(sub_id)
status = subscription_response['status']
print(f"Subscription status: {status}")

subscription_results = list(pl.subscriptions.get_results(sub_id))
print(f"Number of results created by subscription: {len(subscription_results)}")

# Optionally, print the first result to see its structure
if subscription_results:
    print("\nFirst result structure:")
    first_result = subscription_results[0]
    print("Id:  ", first_result['id'])
    print("Status: ", first_result['status'])
    print("Created at: ", first_result['created'])
    print("Full result: ", first_result)
    

Subscription status: completed
Number of results created by subscription: 100

First result structure:
Id:   5b0952c7-9028-4cf7-be2f-4aa9dbc0ba7d
Status:  success
Created at:  2025-06-03T18:50:12.039158Z
Full result:  {'id': '5b0952c7-9028-4cf7-be2f-4aa9dbc0ba7d', 'status': 'success', 'properties': {'item_id': 'BIOMASS-PROXY_V4.0_10_2025-05-09T0000', 'local_solar_time': '2025-05-09T00:00', 'source_id': 'BIOMASS-PROXY_V4.0_10', 'statistics': [{'asset': 'CB', 'band': 'band-1', 'name': 'mean', 'type': 'number', 'value': 0.457}, {'asset': 'CB', 'band': 'band-1', 'name': 'valid_percent', 'type': 'number', 'value': 100}]}, 'created': '2025-06-03T18:50:12.039158Z', 'updated': '2025-06-03T18:50:14.737438Z', 'completed': '2025-06-03T18:50:14.737438Z', 'errors': {}, 'outputs': ['bfdde5ec-f114-475c-9448-c5c580683e00/599624a6-721a-49cf-86f7-373e72e7b26c/2025/05/09/BIOMASS-PROXY_V4.0_10-20250509T0000_CB.tif', 'bfdde5ec-f114-475c-9448-c5c580683e00/599624a6-721a-49cf-86f7-373e72e7b26c/2025/05/09/BIOM

### 7. Subscription data download configuration

##### First check that your configuration works right, and also customize what bands you want to download. 

In [ ]:
# Initialize the API with your configuration
config = Config()

catalog_id = "byoc-" + data_collection_id
# If not already configured, you'll need to set your credentials
config.set("client_id", SENTINEL_HUB_CLIENT_ID)
config.set("client_secret", SENTINEL_HUB_CLIENT_SECRET)
# config.set("instance_id", "your_instance_id")

api = SentinelHubAPI(config)
info = api.get_stac_info(catalog_id)
#info = api.get_collection_info(data_collection_id)
band_info = api.extract_band_info(info)
if band_info:
    print("\nBands:")
    for band_name, band_data in band_info.items():
        description = band_data.get("description", "N/A")
        print(f"  - {band_name}: {description}")
    data_type = api.get_collection_data_type(info)
    print(f"\nDefault Data Type: {data_type}")


### 8. Customize the bands and data type (optional)

The above should report the bands and data type for the collection. If you want to customize the bands or the data type then use the settings below. If they are commented out then the library should automatically download all the bands, using the right data type.

In [15]:
BANDS = ["CB"] # For biomass product
#BANDS = ["SWC"] # For SWC product, other options: ["SWC","SWC_MaskedPixels"] or ["QF","SWC","SWC_MaskedPixels"]  

DATA_TYPE = "uint16"

DOWNLOAD_START_TIME = SUB_START_TIME
DOWNLOAD_END_TIME = SUB_END_TIME
# Can customize these to be different from the subscription start and end times
#DOWNLOAD_START_TIME = "2025-02-25"
#DOWNLOAD_END_TIME = "2025-03-03"

# You can use the code block below to download data for other collection ids. Just uncomment the byoc_id here, comment it out
# in the cell below, and run the cell.
#byoc_id = "a86ee807-1d1b-4f2f-bebf-2a4573dd4c30"
#data_collection_id = "9d4f90e6-34de-4bd2-8437-fca85c868540"


### 9. Data download

In [16]:
# Initialize the API with your configuration
config = Config()
 
# If not already configured, you'll need to set your credentials
config.set("client_id", SENTINEL_HUB_CLIENT_ID)
config.set("client_secret", SENTINEL_HUB_CLIENT_SECRET)
# config.set("instance_id", "your_instance_id")

api = SentinelHubAPI(config)

# Parameters for the download
# Comment out the line below if you want to use a custom collection id from the section above.
byoc_id = data_collection_id  
bbox = None # this will get the bbox from the collection
start_date =  SUB_START_TIME
end_date = SUB_END_TIME
output_dir = OUTPUT_FOLDER  # Where to save the images

if BANDS:
    auto_discover_bands = False
else:
    auto_discover_bands = True  # Automatically detect available bands

# Download the time series
downloaded_files = api.download_byoc_timeseries(
    byoc_id=byoc_id,
    bbox=bbox,
    nodata_value=65535,
    scale_metadata=.001,
    filename_template=OUTPUT_TEMPLATE,
    time_interval=(start_date, end_date),
    output_dir=output_dir,
    auto_discover_bands=auto_discover_bands,
    specified_bands=BANDS,
    data_type=DATA_TYPE
)

print(f"Downloaded {len(downloaded_files)} images to {output_dir}")
for file in downloaded_files:
    print(f"  - {file}")

2025-06-03 15:23:48,293 - sentinelhub_downloader - INFO - Files will be downloaded to: C:\Users\merzoukia.AAFC-AAC\OneDrive - AGR-AGR\Desktop\Test\Planet_PV\downloads\LL_NS_Biomass
2025-06-03 15:24:10,187 - sentinelhub_downloader - INFO - Found 2271 dates with images

Output path: ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-20.tiff
Collection: byoc


Output path: ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-21.tiff
Collection: byoc


Output path: ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-22.tiff
Collection: byoc


Output path: ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-23.tiff
Collection: byoc


...

Output path: ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2025-05-07.tiff
Collection: byoc


Output path: ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2025-05-08.tiff
Collection: byoc


Output path: ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2025-05-09.tiff
Collection: byoc


2025-06-03 16:09:02,931 - sentinelhub_downloader - INFO - Successfully downloaded 2271 of 2271 images to C:\Users\merzoukia.AAFC-AAC\OneDrive - AGR-AGR\Desktop\Test\Planet_PV\downloads\LL_NS_Biomass


Downloaded 2271 images to ./downloads/LL_NS_Biomass
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-20.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-21.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-22.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-23.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-24.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-25.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-26.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-27.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-02-28.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-03-01.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-03-02.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-03-03.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-03-04.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-03-05.tiff
  - ./downloads/LL_NS_Biomass\LL_NS8_Biomass_2019-03-06.tiff
  - ./downloads/LL_NS_Biomass\LL_